# Importing packages

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import requests # request accesing pages
from bs4 import BeautifulSoup # to scrape html code from pages
import re # to cleanup some data points - searches count-
from __future__ import division # needed for division
import pandas as pd # to create a data frame
import numpy as np # generate random times for pauses between requests 
import time # package for pausing code
from time import gmtime, strftime
from pathlib import Path

In [0]:
path = Path('drive/My Drive/AI Australia/')

In [4]:
cd '{path}'

/content/drive/My Drive/AI Australia


In [5]:
ls -al

total 237
-rw------- 1 root root 125157 May  9 05:21  data.csv
-rw------- 1 root root 114650 May  9 05:24 'Scraping jobs from Indeed.ipynb'
-rw------- 1 root root   2367 May  9 05:21  search.csv


# Loading CSVs

In [0]:
#importing searches 

searches = pd.read_csv('search.csv')


# Phase 1: Scraping job links

In [0]:
p1_start = time.time()

# Creating the Data Frame that will hold the scraped data
df_final = pd.DataFrame(columns=["Search", "Title","Location","Company","Salary", "Summary", "Post_Date", "Link"])
print "We are going to scrape jobs for " + str(searches.shape[0]) + " searches:"
print " "
failed = 0
for search in range(0,searches.shape[0], 1):
    df = pd.DataFrame(columns=["Search", "Title","Location","Company","Salary", "Summary", "Post_Date", "Link"])
    url = "https://au.indeed.com/jobs?q=" + searches.job_title[search].lower().replace(' ', '+') + \
    "&l=" + searches.job_location[search].lower().replace(' ', '+') + "&" + 'radius=' + str(searches.search_radius[search]) + "&jt=" + searches.job_type[search] + \
    "&filter=0&limit=50"
    # making sure job title does not have " "
    searches.job_title[search] = searches.job_title[search].replace('"', '')
    
    # request and scrape first page results
    html = requests.get(url)
    soup = BeautifulSoup(html.content, 'html.parser', from_encoding="utf-8")
    
    # scraping the total number of jobs for this search
    try:
        results_count = soup.select('#searchCount')
        # cleaning up the scraped string
        results_count = re.sub('\D', '', str(results_count))[1::]
        results_count = int(results_count)
    except:
        results_count = 'NO'
        failed = failed + 1

    
    print "-------------------------"
    print ""
    if results_count > 1000 and results_count <> "NO":
        print "There are " + str(results_count) +  " jobs for your [  " + str(search + 1) + " / " +  str(str(searches.shape[0])) + "  ] search!, but we can only scrape 1000."
    else:
        print "There are " + str(results_count) +  " jobs for your [  " + str(search + 1) + " / " +  str(str(searches.shape[0])) + "  ] search!"
    
    print ""
    print "link of this search: " + url
    print ""
    

    # to make sure we only scrape the exact number of job posts, otherwise we will have many duplicates

    if results_count < 1000:
        target = results_count
    else:
        target = 1000

    # Loop to request different results pages
    if results_count <> 'NO':

        for page in range(0, target, 50):
            if page is not 0:
                progress =  int(page/target * 100)
                print "Scraped " + str(page) + " jobs [ " + str(progress) + "% ]  [  " + str(search + 1) + " / " +  str(str(searches.shape[0])) + "  ]"

            else:

                print "Initializing Scraping " + str(target) + " " + str(searches.job_title[search]) +" jobs in " + searches.job_location[search] + "!" + \
                "  [  " + str(search + 1) + " / " +  str(str(searches.shape[0])) + "  ]"
                print " "
            new_url = url + "&start=" + str(page)
            html = requests.get(new_url)
            soup = BeautifulSoup(html.content, 'html.parser', from_encoding="utf-8")

        # Loop to scrape job title, location, company, salary, synopsis, post date, link

            for each in soup.find_all(class_= "result"):
                sponsor = each.find(class_='sponsoredGray') 

                if sponsor is None: #ignore any post that is not organic (sponsored)

                    try:
                        date = each.find('span', {'class':'date' }).text.replace('\n', '')
                    except:
                        date = 'None'
                    try:
                        joblink = each.find(class_= 'turnstileLink').get('href')
                        link = 'https://www.indeed.com' + joblink
                    except:
                        link = 'None'
                    try: 
                        title = each.find(class_='jobtitle').text.replace('\n', '')
                    except:
                        title = 'None'
                    try:
                        location = each.find('span', {'class':'location' }).text.replace('\n', '')
                    except:
                        location = 'None'
                    try: 
                        company = each.find(class_='company').text.replace('\n', '')
                    except:
                        company = 'None'
                    try:
                        salary = each.find('span', {'class':'no-wrap'}).text.replace('\n', '')
                    except:
                        salary = 'None'
                    try:
                        summary = each.find('span', {'class':'summary'}).text.replace('\n', '')
                    except:
                        summary = 'None'
        # Append the scraped datapoints into a record in the Data Frame           

                    df = df.append({'Title':title, 'Location':location, 'Company':company,
                                    'Salary':salary, 'Summary':summary, 'Post_Date':date, 'Link':link,
                                    'Search':searches.job_title[search]}, ignore_index=True)

        # Pausing the loop for a random value in a normal distribution with a mean of 1 seconds, imitate human browsing     

            pausetime = int(np.abs(np.random.randn(1) + 1))  
            if page <> 0:
                print "Pausing for " + str(pausetime) + " seconds"
            print "-------------------------"
            time.sleep(pausetime)
    else:
        print "Skipping  [  " + str(search + 1) + " / " +  str(str(searches.shape[0])) + "  ] search!"
    df_final = pd.concat([df_final,df])
    progress =  int(df.shape[0]/target * 100)  

    if results_count <> "NO":
        print " "
        print "Finished Scraping " + str(df.shape[0]) + " jobs [ " + str(progress) + "% ]  [  " + str(search + 1) + " / " +  str(searches.shape[0]) + "  ]  [  Total: " + str(df_final.shape[0])  + " jobs  ]" 
        print " "
    
        if df.shape[0] <> target:
            print "Dont worry if the % or number of jobs dont match, Indeed changes the search results on the go as you browse!"
    
    print " "
print " "
print " "
print "-------------------------"
print "Done with the first phase"
scraped = df_final.shape[0]
if failed <> 0:
    print "There were " + str(failed) + " searches that had no results, so we only scraped jobs for " + str(searches.shape[0] - failed) + " out of the " +  str(searches.shape[0]) + " searches." 
print "Total jobs scraped: " + str(scraped) + " jobs."

df_final.drop_duplicates(keep='first', subset="Link", inplace=True)

duplicates  = scraped - df_final.shape[0]

print "There were " + str(duplicates) + " duplicated jobs that we removed." 
print "So we only saved " + str(df_final.shape[0]) + " jobs."
df_final.reset_index(drop=True, inplace=True)
df1 = df_final

p1_duration = time.strftime("%H:%M:%S", time.gmtime(time.time() - p1_start))
print "Duration: " + p1_duration

We are going to scrape jobs for 18 searches:
 


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


-------------------------

There are 114581616320003199915025010210005100515025010210005100571537339183733466593466444942235659494886670888873676823798070214130563235817962358321661913246620689441621294410687154974442179572836244699510968038254848249922103041030431160282605085831529342271207735044911350480346624572866206622834131467658176824911463113436995181565196331022036825866649660666830040004700200600201140044120157662306466266560087373625666595662667001300340033002005300200340006001400800140027000270014004001440142660729816607668316407702955424046536536536295530536240465365360295542404753653653629554053624046536536370369973000367676737004067300036767137329961070043733300333329963781537439112194075621929104912268622685066310493548720228152232410714130563235817962358321661913246620689441621294410687154974442179572836244699510968038254848249922103041030431160282605085831529342271207735044911350480346625572866206622834131467658176824911463113436995181565197331122036825866649660666830

In [0]:
df_save = df1
df_save.to_csv('data.csv', encoding='utf-8-sig')
!cp data.csv "path", 
